# `nu18` vs `n24`

`n24` is the "Number of children eligible for Child Tax Credit" according to [Tax-Calculator documentation](http://open-source-economics.github.io/Tax-Calculator/). `nu18` is the number of people under age 18.

Based on the Child Tax Credit's [definition of child](https://www.thebalance.com/child-tax-credit-3193009), this should be a subset of children under age 18. One criterion is being under age 17, and others limit further within that.

This notebook examines tax units that violate this assumption by having `n24 > nu18`. This is discussed in [taxdata issue #157](https://github.com/open-source-economics/taxdata/issues/157).

*Data: CPS  |  Tax year: 2014  |  Author: Max Ghenis*

## Setup

### Imports

In [1]:
import taxcalc as tc
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
tc.__version__

'0.19.0'

In [3]:
# Specify number of decimals in tables.
pd.set_option('precision', 2)

## Summaries

In [4]:
recs = tc.Records.cps_constructor()
calc = tc.Calculator(records=recs, policy=tc.Policy())
calc.calc_all()

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


We only care about records `n24 > 0`.

In [5]:
full = calc.dataframe(['s006', 'nu18', 'n24'])

In [6]:
full['nu18_s006'] = full.nu18 * full.s006
full['n24_s006'] = full.n24 * full.s006
full['n24_gt_nu18'] = full.n24 > full.nu18
full['n24_vs_nu18'] = np.where(full.n24 > full.nu18, 'n24 greater',
                               np.where(full.nu18 > full.n24, 
                                        'nu18 greater', 'equal'))

Total `nu18` and `n24`.

In [7]:
full.n24_s006.sum() / 1e6

76.33952

In [8]:
full.nu18_s006.sum() / 1e6

78.647872

In [9]:
full.pivot_table(index='n24_vs_nu18', 
                 values=['s006', 'n24_s006', 'nu18_s006'],
                 aggfunc=sum)

,n24_s006,nu18_s006,s006
n24_vs_nu18,,,
equal,5.39e+07,5.39e+07,1.42e+08
n24 greater,1.40e+07,1.16e+06,8.24e+06
nu18 greater,8.42e+06,2.36e+07,1.32e+07


Limit to `n24>0` for remainder.

In [10]:
df = full[full.n24 > 0]

In [11]:
df.pivot_table(index='n24_gt_nu18', 
               values=['s006', 'n24_s006', 'nu18_s006'],
               aggfunc=sum)

,n24_s006,nu18_s006,s006
n24_gt_nu18,,,
False,6.24e+07,6.78e+07,3.57e+07
True,1.40e+07,1.16e+06,8.24e+06


Drill into records where `n24>nu18` by `nu18`.

In [12]:
df[df.n24_gt_nu18].pivot_table(
    index='nu18', 
    values=['s006', 'n24_s006', 'nu18_s006'],
    aggfunc=sum)

,n24_s006,nu18_s006,s006
nu18,,,
0.0,1.12e+07,0.00e+00,7.18e+06
1.0,2.52e+06,1.01e+06,1.01e+06
2.0,9.39e+04,5.34e+04,2.67e+04
3.0,4.26e+04,3.10e+04,1.03e+04
4.0,8.38e+04,6.70e+04,1.68e+04


In [13]:
df[df.n24_gt_nu18].pivot_table(
    index='nu18', 
    columns='n24',
    values=['s006', 'n24_s006', 'nu18_s006'],
    aggfunc=sum)

n24_s006                                         nu18_s006            \
n24        1.0       2.0       3.0       4.0       5.0       1.0       2.0   
nu18                                                                         
0.0   4.31e+06  3.95e+06  1.96e+06  697956.0  324815.0       0.0       0.0   
1.0        NaN  1.32e+06  6.82e+05  361224.0  157745.0       NaN  658109.0   
2.0        NaN       NaN  5.05e+04   23680.0   19700.0       NaN       NaN   
3.0        NaN       NaN       NaN   36104.0    6500.0       NaN       NaN   
4.0        NaN       NaN       NaN       NaN   83780.0       NaN       NaN   

                                      s006                                \
n24        3.0      4.0      5.0       1.0       2.0       3.0       4.0   
nu18                                                                       
0.0        0.0      0.0      0.0  4.31e+06  1.97e+06  654623.0  174489.0   
1.0   227218.0  90306.0  31549.0       NaN  6.58e+05  227218.0   90306.0   
2.0    33654.0  11840.0   7880.0       NaN       NaN   16827.0    5920.0   
3.0        NaN  27078.0   3900.0       NaN       NaN       NaN    9026.0   
4.0        NaN      NaN  67024.0       NaN       NaN       NaN       NaN   

               
n24       5.0  
nu18           
0.0   64963.0  
1.0   31549.0  
2.0    3940.0  
3.0    1300.0  
4.0   16756.0

Share of tax units with `n24>0` where `n24>nu18`.

In [14]:
df.loc[df.n24_gt_nu18, 's006'].sum() / df.s006.sum()

0.18737077393125368